# Task 3: Helper notebook for loading the data and saving the predictions

In [1]:
import pickle
import gzip
import numpy as np
import os

In [2]:
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from skimage.transform import resize

from keras.preprocessing.image import load_img
from keras import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.optimizers import adam_v2
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout, BatchNormalization
from keras import backend as K

import cv2


### Helper functions

In [3]:
def build_model_normalization(input_layer, start_neurons):
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(input_layer)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(0.25)(pool1)

    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(0.5)(pool2)

    conv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)
    pool3 = Dropout(0.5)(pool3)

    conv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(conv4)
    conv4 = BatchNormalization()(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(0.5)(pool4)

    # Middle
    convm = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(pool4)
    convm = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(convm)
    
    deconv4 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(0.5)(uconv4)
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(uconv4)
    uconv4 = BatchNormalization()(uconv4)
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(uconv4)
    uconv4 = BatchNormalization()(uconv4)

    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv4)
    uconv3 = concatenate([deconv3, conv3])
    uconv3 = Dropout(0.5)(uconv3)
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(uconv3)
    uconv3 = BatchNormalization()(uconv3)
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(uconv3)
    uconv3 = BatchNormalization()(uconv3)

    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Dropout(0.5)(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = BatchNormalization()(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = BatchNormalization()(uconv2)

    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Dropout(0.5)(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = BatchNormalization()(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = BatchNormalization()(uconv1)
    
    output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv1)
    
    return output_layer

In [4]:
def build_model(input_layer, start_neurons):
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(input_layer)
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    pool1 = Dropout(0.25)(pool1)

    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    pool2 = Dropout(0.5)(pool2)

    conv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(pool2)
    conv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)
    pool3 = Dropout(0.5)(pool3)

    conv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(pool3)
    conv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)
    pool4 = Dropout(0.5)(pool4)

    # Middle
    convm = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(pool4)
    convm = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(convm)
    
    deconv4 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(0.5)(uconv4)
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(uconv4)
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(uconv4)

    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv4)
    uconv3 = concatenate([deconv3, conv3])
    uconv3 = Dropout(0.5)(uconv3)
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(uconv3)
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(uconv3)

    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Dropout(0.5)(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)

    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Dropout(0.5)(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    
    output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv1)
    
    return output_layer

In [5]:
def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        loaded_object = pickle.load(f)
        return loaded_object

In [6]:
def save_zipped_pickle(obj, filename):
    with gzip.open(filename, 'wb') as f:
        pickle.dump(obj, f, 2)

In [7]:
def evaluate(predictions, targets):
    ious = []
    for p, t in zip(predictions, targets):
        assert p['name'] == t['name']
        prediction = np.array(p['prediction'], dtype=bool)
        target = np.array(t['label'], dtype=bool)

        assert target.shape == prediction.shape
        overlap = prediction*target
        union = prediction + target

        ious.append(overlap.sum()/float(union.sum()))
    
    print("Median IOU: ", np.median(ious))

In [8]:
def preprocess_train(data, img_size_target):
    X_fin, y_fin = [], []
    video = data['video']
    frames = data['frames']
    label = data['label']
    h, w = video.shape[0], video.shape[1]
    X = video[:,:,frames]
    y = label[:,:,frames] * 1
    X = cv2.resize(X, (img_size_target, img_size_target), interpolation = cv2.INTER_NEAREST)
    y = cv2.resize(y, (img_size_target, img_size_target), interpolation = cv2.INTER_NEAREST)
    y = (y > 0) * 1.0
    for i in range(len(frames)):
        X_fin.append(X[:,:,i])
        y_fin.append(y[:,:,i])
    return X_fin, y_fin

In [9]:
def preprocess_test(data, img_size_target):
    X_fin = []
    video = data['video']
    for i in range(video.shape[2]):
        frame = cv2.resize(video[:,:,i], (img_size_target, img_size_target), interpolation = cv2.INTER_NEAREST)
        X_fin.append(frame)
    return X_fin

In [10]:
def jaccard_similarity(y_train, y_test):
    y_train_f = K.flatten(y_train)
    y_test_f = K.flatten(y_test)
    intersection = K.sum(y_train_f * y_test_f)
    union = K.sum(y_train_f) + K.sum(y_test_f) - intersection
    return intersection / union

In [11]:
def jaccard_loss(y_train, y_test):
    return - jaccard_similarity(y_train, y_test)

### Load data and Initiate variables

In [12]:
# load data
train_data = load_zipped_pickle("train.pkl")
test_data = load_zipped_pickle("test.pkl")
samples = load_zipped_pickle("sample.pkl")

In [13]:
img_size_target = 112
train = train_data
test = test_data

### Detection

In [14]:
X, y = [], []
for data in train:
    X_new, y_new = preprocess_train(data, img_size_target)
    X += X_new
    y += y_new
X = np.array(X)
y = np.array(y)

In [15]:
X.shape

(195, 112, 112)

In [16]:
n = 3 * 10
X_train = X[:-n,:,:]
X_val = X[-n:,:,:]
y_train = y[:-n,:,:]
y_val = y[-n:,:,:]

In [17]:
X_train = np.expand_dims(X_train, 3)
y_train = np.expand_dims(y_train, 3)
X_val = np.expand_dims(X_val, 3)
y_val = np.expand_dims(y_val, 3)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(165, 112, 112, 1)
(165, 112, 112, 1)
(30, 112, 112, 1)
(30, 112, 112, 1)


In [29]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
datagen.fit(X_train)

In [30]:
input_layer = Input((img_size_target, img_size_target, 1))
output_layer = build_model(input_layer, 16)

In [31]:
model = Model(input_layer, output_layer)

In [32]:
model.compile(loss=jaccard_loss, optimizer="adam", metrics=[jaccard_similarity, 'accuracy'])

In [33]:
epochs = 20
batch_size = 32

early_stopping = EarlyStopping(patience=25, verbose=1)
model_checkpoint = ModelCheckpoint("./keras.model", save_best_only=True, verbose=1)


history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=datagen.flow(X_val, y_val, batch_size=32), 
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=[early_stopping, model_checkpoint])

Epoch 1/20
6/6 [==============================] - ETA: 0s - loss: -0.0092 - jaccard_similarity: 0.0092 - accuracy: 0.2287
Epoch 00001: val_loss improved from inf to -0.00754, saving model to .\keras.model
INFO:tensorflow:Assets written to: .\keras.model\assets
6/6 [==============================] - 11s 2s/step - loss: -0.0092 - jaccard_similarity: 0.0092 - accuracy: 0.2287 - val_loss: -0.0075 - val_jaccard_similarity: 0.0075 - val_accuracy: 0.0075
Epoch 2/20
6/6 [==============================] - ETA: 0s - loss: -0.0097 - jaccard_similarity: 0.0100 - accuracy: 0.0118
Epoch 00002: val_loss improved from -0.00754 to -0.00859, saving model to .\keras.model
INFO:tensorflow:Assets written to: .\keras.model\assets
6/6 [==============================] - 10s 2s/step - loss: -0.0097 - jaccard_similarity: 0.0100 - accuracy: 0.0118 - val_loss: -0.0086 - val_jaccard_similarity: 0.0086 - val_accuracy: 0.0073
Epoch 3/20
6/6 [==============================] - ETA: 0s - loss: -0.0106 - jaccard_similar

KeyboardInterrupt: 

In [24]:
predictions = []
for data in test:
    prediction = np.array(np.zeros_like(data['video']), dtype=np.bool)
    X_new = preprocess_test(data, img_size_target)
    print(len(X_new))
    for i, frame in enumerate(X_new):
        plt.show()
        pred = model.predict(frame.reshape(1,img_size_target,img_size_target))
        pred = pred > 0.1
        pred = pred * 1
        prediction[:,:,i] = cv2.resize(pred[0,:,:,0], (data['video'].shape[1], data['video'].shape[0]), interpolation = cv2.INTER_NEAREST) > 0
    predictions.append({
        'name': data['name'],
        'prediction': prediction
        }
    )

84
78
53
104
73
159
72
63
90
62
94
68
51
63
125
65
54
106
39
69


In [25]:
predictions

[{'name': 'RZ9W7OK2EO',
  'prediction': array([[[False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          ...,
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False]],
  
         [[False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          ...,
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False]],
  
         [[False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          ...,
          [False, False,

In [26]:
# save in correct format
save_zipped_pickle(predictions, 'my_predictions.pkl')

# Useless cells I might use

In [ ]:
input_layer = Input((img_size_target, img_size_target, 1))
output_layer = build_model(input_layer, 16)

In [ ]:
model = Model(input_layer, output_layer)

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
X, y = [], []
for data in train:
    X_new, y_new = preprocess_train(data, img_size_target)
    X += X_new
    y += y_new
X = np.array(X)
y = np.array(y)

In [ ]:
X[:-3].shape

In [ ]:
epochs = 200
batch_size = 32

early_stopping = EarlyStopping(patience=10, verbose=1)
model_checkpoint = ModelCheckpoint("./keras.model", save_best_only=True, verbose=1)


history = model.fit(X[:-3], y[:-3],
                    validation_data=[X[-3:], y[-3:]], 
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=[early_stopping, model_checkpoint])


In [ ]:
res = model.predict(X[-5:])
res

In [ ]:
res2 = res > 0.95
res2

In [ ]:
plt.imshow(res2[-2,:,:,0])

In [ ]:
plt.imshow(y[-2])

In [ ]:
A, B = res2[-2,:,:,0], y[-2]

In [ ]:
np.sum(np.multiply(A, B))/np.sum(A+B>0)

In [ ]:
# make prediction for test
predictions = []
for d in test_data:
    prediction = np.array(np.zeros_like(d['video']), dtype=np.bool)
    height = prediction.shape[0]
    width = prediction.shape[1]
    prediction[int(height/2)-50:int(height/2+50), int(width/2)-50:int(width/2+50)] = True
    
    # DATA Strucure
    predictions.append({
        'name': d['name'],
        'prediction': prediction
        }
    )

In [ ]:
plt.imshow(X[4])

In [ ]:
plt.imshow(y[4])

In [ ]:
train_sample = train_data[41]

In [ ]:
train_sample

In [ ]:
video = train_sample['video']

In [ ]:
label = train_sample['label']

In [ ]:
video.shape

In [ ]:
# minx, miny, maxx, maxy = 1, 1, 0, 0
# for train_example in train:
#     box = train_example['box']
#     video = train_example['video']
#     h = video.shape[0]
#     w = video.shape[1]
#     x, y = np.where(box)
#     minx, miny, maxx, maxy = min(x[0]/h, minx), min(y[0]/w, miny), max(x[-1]/h, maxx), max(y[-1]/w, maxy)
# print(minx, miny, maxx, maxy)

In [ ]:
# minx, miny, maxx, maxy = 0.35, 0.45, 0.85, 0.7

In [ ]:
box = train_sample['box']

In [ ]:
np.where(box)

In [ ]:
train[-1]

In [ ]:
plt.imshow(train[-1]['video'][:,:,-1])

In [ ]:
pl

In [ ]:
train_sample['frames']

In [ ]:
np.where(label)

In [ ]:
pred = predictions[5]['prediction']
pred

In [ ]:
pred.shape

In [ ]:
plt.imshow(pred[:,:,5])

In [ ]:
test = test_data[3]
test

In [ ]:
test['video'].shape

In [ ]:
for i in range(len(y_train)):
    plt.imshow(y_train[i,:,:])
    plt.show()

In [ ]:
%matplotlib inline
import matplotlib
import ipywidgets
from PIL import Image
matplotlib.rcParams.update({'figure.max_open_warning': 0}) #avoids a warning about how many figures to keep in memory


In [ ]:
def display_simulation():
    global FIGS
    slider = ipywidgets.IntSlider(value=0,
                                  min=0,
                                  max=len(FIGS)-2)

    def figshow(i):
        return display(FIGS[i])

    ipywidgets.interact(figshow,i=slider);

In [ ]:
FIGS = [Image.fromarray(y_train[i,:,:]) for i in range(len(y_train))]

In [ ]:
display_simulation()